In [14]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [16]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
# dataset loaded
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [17]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        #w =["how", "old", "are", "you"] 
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [18]:
# lemmaztize and lower each word and remove duplicates
#Happy happy
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

90 documents
28 classes ['address', 'be', 'branches', 'college_name', 'comp_shifts', 'comps_intake', 'computer', 'courses', 'diploma', 'district', 'documents', 'elect_intake', 'elect_shifts', 'electrical', 'electronic', 'extc', 'extc_ahifts', 'extc_intake', 'goodbye', 'greeting', 'it', 'me', 'mech_intake', 'mech_shifts', 'mechanical', 'phd', 'thanks', 'what_help']
79 unique lemmatized words ["'s", ',', 'address', 'admission', 'anyone', 'are', 'awesome', 'bachelor', 'be', 'branch', 'bye', 'can', 'celectrical', 'chatting', 'college', 'computer', 'could', 'course', 'day', 'different', 'diploma', 'district', 'do', 'document', 'electrical', 'electronic', 'engineering', 'extc', 'for', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'in', 'intake', 'is', 'it', 'later', 'located', 'location', 'many', 'master', 'me', 'mechanical', 'much', 'name', 'next', 'nice', 'of', 'offered', 'phd', 'provide', 'require', 'required', 'see', 'shift', 'support', 'ta

In [19]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [20]:
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'what_help'), (['What', 'you', 'can', 'do', '?'], 'what_help'), (['What', 'help', 'you', 'provide', '?'], 'what_help'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'what_help'), (['What', 'support', 'is', 'offered'], 'what_help'), (['What', 'is', 'college', 'name', '?'], 'college_name'), (['What', 'is', 'name', 'of', 'the', 'college', '?'], 'college_name'), (['Could'

In [21]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence

In [22]:
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [23]:
print(bag)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]


In [24]:
print(output_row)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [25]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [26]:
print("All the inputs/patterns in binary")
print("")
print(train_x)

All the inputs/patterns in binary

[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0

In [27]:
print("All the outputs in binary")
print("")
print(train_y)

All the outputs in binary

[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0

In [29]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [32]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               10240     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 28)                1820      
Total params: 20,316
Trainable params: 20,316
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1971 - accuracy: 0.9444
Epoch 2/200
18/18 [==============================] - 0s 1ms/step - loss: 0.1691 - accuracy: 0.9444
Epoch 3/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1227 - accuracy: 0.9444
Epoch 4/200
18/18 [==============================] - 0s 1ms/step - loss: 0.1784 - accuracy: 0.9444
Epoch 5/200
18/18 [==============================] - 0s 1ms/step - loss: 0.1224 - accuracy: 0.9444
Epoch 6/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1793 - accuracy: 0.9111
Epoch 7/200
18/18 [==============================] - 0s 2ms/step - loss: 0.2206 - accuracy: 0.9222
Epoch 8/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0961 - accuracy: 0.9667
Epoch 9/200
18/18 [==============================] - 0s 1ms/step - loss: 0.1772 - accuracy: 0.9222
Epoch 10/200
18/18 [==============================] - 0s 1ms/step - loss: 0.1192 - accuracy: 0.9667
Epoch 11/

18/18 [==============================] - 0s 2ms/step - loss: 0.0950 - accuracy: 0.9667
Epoch 84/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1637 - accuracy: 0.9333
Epoch 85/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1184 - accuracy: 0.9333
Epoch 86/200
18/18 [==============================] - 0s 2ms/step - loss: 0.0795 - accuracy: 0.9778
Epoch 87/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1058 - accuracy: 0.9556
Epoch 88/200
18/18 [==============================] - 0s 2ms/step - loss: 0.0946 - accuracy: 0.9556
Epoch 89/200
18/18 [==============================] - 0s 2ms/step - loss: 0.0634 - accuracy: 0.9667
Epoch 90/200
18/18 [==============================] - 0s 2ms/step - loss: 0.0711 - accuracy: 0.9778
Epoch 91/200
18/18 [==============================] - 0s 2ms/step - loss: 0.0809 - accuracy: 0.9667
Epoch 92/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1326 - accuracy: 0.9556
Epoch 93/200
